## 11. Архитектура программного обеспечения

> «Хорошая архитектура отвечает потребностям пользователей, разработчиков и владельцев не только сейчас, но и продолжит отвечать им в будущем».  
>  
> Роберт Мартин, «Чистая архитектура».  

![Architecture](https://raw.githubusercontent.com/amaargiru/pycore/main/pics/11_Architecture.png)  
[Картинка в большом разрешении](https://raw.githubusercontent.com/amaargiru/pycore/main/pics/11_Architecture.png)  

### Что такое архитектура 

Во вопросу точного значения термина «Архитектура программного обеспечения» было сломано много копий, но точного (и притом общепринятого) ответа пока не найдено. Что касается моего личного мнения, то, на мой взгляд, ПО, построенное по принципам «хорошей» архитектуры, должно удовлетворять всего двум требованиям:  
1. выполнять требования ТЗ в настоящем;  
2. спокойно и легко реагировать на изменения ТЗ в будущем.

Для этого, в свою очередь, ПО должно быть:  
1. правильно разрезано на кусочки (правильно фрагментировано на зоны ответственности, файлы, классы и микросервисы) — целостные, со сфокусированной зоной ответственности, понятные и относительно простые;  
2. правильно соединено (должны быть определены общие структуры данных, интерфейсы и механизмы передачи данных между фрагментами).

А вот три архитектурных требования из книги Мартина Клеппмана "Высоконагруженные приложения" (или как её часто называют в русскоязычном сегменте интернета, "Книги с кабанчиком"):  
1. надежность - система должна продолжать работать корректно (осуществлять нужные функции на требуемом уровне производительности) даже при неблагоприятных обстоятельствах (в случае аппаратных или программных сбоев либо ошибок пользователя);
2. масштабируемость - должны быть предусмотрены разумные способы решения возникающих при росте (в смысле объемов данных, трафика или сложности) системы проблем;  
3. удобство сопровождения - необходимо обеспечить возможность эффективной работы с системой множеству различных людей (разработчикам и обслуживающему персоналу, занимающимся как поддержкой текущего функционирования, так и адаптацией системы к новым сценариям применения).  

### Чистая архитектура

Обычно принципы действия работы чистой архитектуры принято разбирать, основываясь на симпатичной круговой диаграмме (ранее использовался даже термин Onion Architecture, ныне подзабытый), продвигаясь от центра к внешним границам и разъясняя трансформации, происходящие по пути. Круговые диаграммы выглядят эстетично и вселяют какую-то особую надежду на своё непорочное совершенство и непоколебимую устойчивость. На части иллюстраций многослойный кружочек еще и разрезают, выворачивая его сначала в полукружие, а потом и в линейную нарезку.

Давайте попробуем всё немного упростить и обойтись вовсе без графических иллюстраций.

Для этого рассмотрим сначала обратный пример — «грязную» архитектуру.

Итак, мы хотим создать совершенно новую программу. Не будем усложнять пример, мы хотим написать не картографический сервис, конкурент Google Maps и Яндекс Карт, а несложную программу-игрушку «Крестики-Нолики» на C# с простейшим CLI (Command Line Interface), когда ввод хода пользователя и вывод результирующего игрового поля происходит не при помощи графического интерфейса, а с помощью скромного bash. Коротенькая программа написана, хорошо работает и радует своего владельца.

Программист, который разработал «Крестики-Нолики», был человеком разносторонним и решил переделать интерфейс программы с CLI на WPF. Переделка заняла не очень много времени и очень скоро новая программа в Windows-стиле радовала своего создателя. Еще через некоторое время разработчик при помощи Blazor сделал себе такую же игрушку, но с web-интерфейсом, чтобы можно было скоротать время при помощи браузера.

Спустя некоторое время программист обнаружил небольшую ошибку в коде программы. При некоторых, нечастых комбинациях игрового поля игра вела себя некорректно и выдавала неверный результат. К счастью, ошибка была простой и её исправление во всех трех программах с тремя разными интерфейсами (CLI, оконный и web) не заняло много времени.

Какие основные отрицательные моменты у продемонстрированного подхода, у «грязной» архитектуры?

Первое — логика самой игры перемешана с логикой интерфейса. При смене интерфейса (сущностью, *внешней* относительно игрового движка) необходимо переделывать *всё*.
Второе — при расширении функционала программы, при исправлении ошибок, даже при простом рефакторинге нужно править некоторые *одинаковые* сущности в *нескольких* местах, что, разумеется, никак не способствует ни скорости, ни качеству работы.
Третье — ядро игры, её логика, *вынуждена знать*, как работают те или иные интерфейсы, просто для нормального взаимодействия с ними, ведь это именно ядро решает, что увидит пользователь в рамках того или иного интерфейса в каждый конкретный момент игры.

Если эти небольшие, на первый взгляд, шероховатости умножить на количество сущностей, эксплуатируемых в рамках одного современного программного продукта и экстраполировать сложность с пет-проекта на реальный бизнес-продукт, то все указанные проблемы вполне могут привести к тому, что существующий проект будет очень дорого или почти невозможно корректировать и модифицировать.

Перейдем к, собственно, чистой архитектуре. Напомню, что чистая архитектура — не панацея и не лекарство от всех проблем. Есть множество вопросов, на которые она отнюдь не даёт ответ, и проектирование ПО с помощью подходов чистой архитектуры вовсе не превращается в формальную пошаговую процедуру. Хотя чистая архитектура декларирует, что сеть и БД являются всего лишь внешним окружением и имеют второстепенную важность, вы, скорее всего, не сможете быстренько заменить PostgreSQL на Redis, а REST на gRPC, если вдруг внезапно этого захотите. Но дополнительные слои абстракций вы получите гарантированно.

Проектирование нашей игры «Крестики-Нолики» чистая архитектура рекомендует начать с выделения так называемой *бизнес-логики*. Бизнес-логика, если по-простому — эта та часть нашего проекта, которая будет сохранена, даже если на свете не будет ни сетей связи, ни баз данных, ни даже компьютеров. Банковский бизнес, торговля, образование, картография и тысячи других придумок человечества (включая, кстати, «Крестики-Нолики») прекрасно работали без CPU, БД и REST, используя бумагу, пергамент, козьи шкуры или клинопись по сырой глине.

Бизнес-логика ничего не знает о внешнем мире, использует удобные именно ей имена переменных, DTO, вычислительные схемы и структуры данных. Бизнес-логика никогда не обращается напрямую к базе данных или к сети, она просто не имеет ни о базах, ни о сетях ни малейшего понятия. Всё, что знает бизнес-логика игры «Крестики-Нолики» — это правила игры в «Крестики-Нолики».

Далее идут так называемые сценарии использования (use cases) — описание поведения системы при взаимодействии с внешними сущностями (или акторами), которые могут быть как людьми, так и другими программами. Сценарии, в свою очередь, знают о бизнес-логике, используют её, но ничего не знают о слое, находящимся выше. Нужно помнить, что сценарии использования сейчас не так актуальны, как раньше, и вместе с UML, RUP и другими сущностями, призванными упростить бизнес-планирование в программировании, потихоньку теряют свою актуальность. Так что вы можете не разделять бизнес-логику и сценарии.

Далее лежит слой интерфейсов. В этом слое происходит преобразование информации из формата, удобного для бизнес-логики и сценариев использования, в формат, удобный для внешних сущностей, таких, как база данных или пользовательский интерфейс.

И, наконец, самый внешний слой — связующее звено с внешним миром, с фреймворками, интерфейсами и базами данных.

Общее количество слоёв может меняться, главное — всегда должно выполняться правило зависимостей: зависимости всегда должны быть направлены внутрь.

### SOLID 

SOLID — название методики и акроним для пяти основных принципов проектирования программного обеспечения на базе объектно-ориентированного подхода. Как и применение других архитектурных принципов, применение SOLID помогает поддерживать и модернизировать код.

Принципы SOLID подсказывают, как лучше объединять функции и структуры данных в классы и лучшие пути взаимодействия между классами. SOLID также можно использовать в качестве ориентира в процессе рефакторинга кода. SOLID применим на уровне модулей, т. н. «среднем уровне», т. е. выше уровня собственно кода, но ниже высокоуровневых архитектурных принципов.

**SRP**

Single-responsibility principle, принцип единственной ответственности.  
Предполагает проектирование классов, имеющих только одну причину для изменения, позволяет вести проектирование в направлении, противоположном созданию «[Божественных объектов](https://en.wikipedia.org/wiki/God_object)». Окончательная версия SRP формулируется как «Модуль должен отвечать за одного и только за одного актора»; сам Роберт Мартин считал SRP несколько трудным для понимания, но на мой взгляд, всё достаточно сильно упрощается, если вместо «акторов» использовать термин «группа лиц». Тогда сразу становится понятно, что «актор» — это не какие-то взаимосвязанные классы или иные программные сущности, а всего лишь люди, пользователи, в интересах которых и проводится разработка программы. Соответственно, класс должен отвечать за удовлетворение запросов только одной группы лиц.

Если еще больше упростить формулировку, то можно даже сказать, что класс должен отвечать за интересы только одного отдела внутри организации. Соответственно, если класс реализует функционал, необходимый одновременно и бухгалтерии, и снабженцам или, например, и конструкторам, и технологам - такой класс необходимо переписать, разделив на два.

**OCP**

Open–closed principle, принцип открытости/закрытости.  
Классы должны быть закрыты от изменения (чтобы код, опирающийся на эти классы, не нуждался в обновлении), но открыты для расширения (классу можно добавить новое поведение). Вкратце — хочешь изменить поведение класса — оставь в неприкосновенности старый код (не считая рефакторинга, т. е. изменение программы без изменения внешнего поведения), добавь новый. Если расширение требований ведет к значительным изменениям в существующем коде, значит, были допущены архитектурные ошибки.

**LSP**

Liskov substitution principle, принцип подстановки Барбары Лисков.  
Поведение наследующих классов должно быть ожидаемым для кода, использующего переменную базового класса. Или, другими словами, подкласс не должен требовать от вызывающего кода больше, чем базовый класс, и не должен предоставлять вызывающему коду меньше, чем базовый класс.

**ISP**

Interface segregation principle, принцип разделения интерфейса.  
Клиент интерфейса не должен зависеть от неиспользуемых методов. В соответствии с принципом ISP рекомендуется создавать минималистичные интерфейсы, содержащие минимальное количество специфичных методов. Если пользователь интерфейса не пользуется каким-либо методом интерфейса, то лучше создать новый интерфейс, без этого метода.

**DIP**

Dependency inversion principle, принцип инверсии зависимостей.  
Модули верхнего уровня не должны обращаться к модулям нижнего уровня напрямую, между ними должна быть «прокладка» из абстракций (т. е. интерфейсов). Причем абстракции не должны зависеть от реализаций, реализации должны зависеть от абстракций.

### Связность кода и code coupling

<img src="coupling_vs_cohesion.svg" style="height:320px">

Связность кода (cohesion) характеризует целостность или «фокусировку» программного модуля, т.е. то, насколько внутренние компоненты модуля логически связанны друг с другом.

Сцепление модулей (coupling) — степень взаимной зависимости разных программных модулей. Сильное зацепление — серьёзная проблема, т. к. затрудняет переиспользование, автономное тестирование, и даже просто модификацию отдельных модулей. Фактически, в условиях сильного зацепления приходится рассматривать всю проектируемую систему как один модуль, что, естественно ведет к росту когнитивной нагрузки, падению производительности и росту количества ошибок.

Хорошо спроектированная система характеризуется высокой связностью и низким сцеплением.

### KISS

Keep it simple, stupid — принцип проектирования ПО, в соответствии с которым простота системы декларируется как одна из основополагающих ценностей (иногда даже простота объявляется более важной, чем любые другие свойства системы, см. [Worse is Better](https://en.wikipedia.org/wiki/Worse_is_better) Ричарда Гэбриела), одно из практических приложений «[Бритвы Оккама](https://en.wikipedia.org/wiki/Occam%27s_razor)» — не создавай новых сущностей без крайней необходимости.

### DRY

Don’t repeat yourself (не повторяйся) — принцип, в соответствии с которым изменение любого элемента системы не должно требовать внесения изменений в другие, логически не связанные элементы. Логически же связанные элементы изменяются предсказуемо и единообразно.

### YAGNI

You aren't gonna need it (вам это не понадобится) — если в определенном функционале нет потребности прямо здесь и прямо сейчас — не добавляй его. Этим ты не только отнимешь время, необходимое на разработку и тестирование действительного функционала, но и можешь подложить себе мину замедленного действия на будущее, когда контуры развития системы станут более четкими.

### Парадигмы программирования

[Процедурное программирование](https://en.wikipedia.org/wiki/Procedural_programming)

Методика, в соответствии с которой последовательно выполняемые операторы собирают в подпрограммы, то есть более крупные целостные единицы кода.

[Структурное программирование](https://en.wikipedia.org/wiki/Structured_programming)

Понимание того, что можно исключить оператор goto и представить программу в виде иерархической структуры блоков (из трёх базовых управляющих конструкций: последовательность, ветвление, цикл).

Концепция структурного программирование получила широкое распространение примерно в 1970-х, когда было строго математически доказано, что на базе последовательности, ветвления и цикла можно написать любую, сколь угодно сложную программу, а [Эдсгер Дейкстра](https://en.wikipedia.org/wiki/Edsger_W._Dijkstra) даже написал в 1968 статью «О вреде оператора goto» («A Case against the GO TO Statement»).

На всякий случай, если вы программируете на языке ассемблера и широко используете условные и безусловные переходы, это совершенно нормально. Goto и jmp соотносятся примерно никак.  
jmp — благородная ассемблерная инструкция, ветвящая исполнение программы.  
goto — костыль, позволяющий начинающему программисту внезапно телепортироваться в другое место кода.  
Объединяет их только то, что goto после компиляции превращается в jmp, ну так и for, и if тоже превращаются в jmp.

[Объектно-ориентированное программирование](https://en.wikipedia.org/wiki/Object-oriented_programming)

Методология, основанная на представлении программы в виде совокупности взаимодействующих объектов, каждый из которых является экземпляром определённого класса, а классы образуют иерархию наследования.

[Функциональное программирование](https://en.wikipedia.org/wiki/Functional_programming)

Программирование на базе математических функций. Математические функции не являются методами в программном смысле, их лучше всего рассматривать как канал (pipe), преобразующий любое значение, которое мы передаем, в другое значение.

Программный метод становится математической функцией после выполнения двух требований:  
1. Он должен быть ссылочно прозрачным (referentially transparent). Ссылочно прозрачная функция всегда дает один и тот же результат, если вы предоставляете ей одни и те же аргументы. Это означает, что такая функция должна работать только со значениями, которые мы передаем, она не должна ссылаться ни на какие глобальное состояния.  
2. Сигнатура метода должна передавать всю информацию о возможных принимаемых входных значениях, и о возможных результатах, которые он выдает.  

Преимущество функционального программирования — относительная простота, так как каждый метод, выполняющий два требования, указанных выше, можно рассматривать и отлаживать в условиях полной изоляции от остальной кодовой базы. Плюс, иммутабельность позволяет компилятору применять более агрессивные методы оптимизации.

[Другие парадигмы](https://en.wikipedia.org/wiki/Programming_paradigm)

### Основные принципы ООП

[Наследование](https://en.wikipedia.org/wiki/Inheritance_(object-oriented_programming))  

Способность компонента базироваться на другом компоненте. Можно создать общий класс, который будет определять характеристики и поведение, свойственные набору связанных объектов.

[Инкапсуляция](https://en.wikipedia.org/wiki/Encapsulation_(computer_programming))  

Сокрытие внутренних данных компонента и деталей его реализации от других компонентов приложения и предоставление набора методов для взаимодействия с ним (API).

[Полиморфизм](https://en.wikipedia.org/wiki/Polymorphism_(computer_science))

Способность компонента выбирать внутреннюю процедуру (метод) исходя из типа данных, принятых в сообщении. Следует разделять ad hoc полиморфизм и параметрический полиморфизм.

[Абстракция](https://en.wikipedia.org/wiki/Abstraction#In_computer_science)

Представление объекта минимальным набором данных и методов с точностью, достаточной для решаемой задачи.

### Принцип CQRS

CQRS (Command & Query Responsibility Segregation) — принцип разделения запросов на чтение данных и команд на изменение данных. В соответствии с принципом CQRS не должно быть запросов, одновременно и изменяющих, и считывающих данные. Или иначе — данные не должны модифицироваться во время подготовки ответа на запрос; формирование ответа не должно иметь побочных эффектов.

Применение принципа CQRS позволяет упростить программу, так как код становится более читаемым, отладка - более прозрачной, предсказуемой и повторяемой.

Сам по себе CQRS не является глубокомысленной архитектурной парадигмой, это всего лишь достаточно простой шаблон, применение которого хорошо сочетается с большим количеством самых разнообразных архитектур. Конкретная имплементация принципа зависит от архитектуры компонентов, используемых в системе, и может вылиться, например, в разделение очередей запросов данных и команд изменения данных.

Хотя CQRS является не какой-то самодостаточной архитектурной сущностью, а скорее концепцией или «советом» по поводу общих принципов проектирования, тем не менее, вдумчивое применение CQRS способно привести к достаточно глубокому изменению архитектурных принципов построения приложения.

Рассмотрим, например, следующий пример. Скажем, у нас есть сайт, отображающий курсы валют, бэкенд бизнес-логика которого имеет следующие методы:

```python
# CurrencyInterface

def GetCurrency(CurrencyID) -> Currency:  # Работа с одиночным значением
def SetCurrency(CurrencyID, Currency) -> None:

def GetCurrencyList(CurrencyID, datetime start, datetime end) -> List[Currency]:  # Работа с последовательностью данных
def SetCurrencyList(List[Currency]) -> None:
```

Вроде всё выглядит «нормально» — мы можем читать и писать как одиночные значения, так и последовательности данных. На первый взгляд, данные на запись и на чтение — это, можно сказать, одни и те же данные, и работа с ними выглядит достаточно консистентно.

Но, предположим, сайт стал набирать популярность и перед нами стала задача масштабирования архитектуры бэкенда. Сайт очень популярен, и мы развернули бэкенд аж на десяти серверах, каждый из которых, естественно, поддерживает как запись, так и чтение данных.

Что мы сделали не так? Чтобы разобраться, проведем еще один мысленный эксперимент. Сделаем *два* интерфейса:

```python
# ReadCurrencyInterface

def GetCurrency(CurrencyID) -> Currency:
def GetCurrencyList(CurrencyID, datetime start, datetime end) -> List[Currency]:

# WriteCurrencyInterface

def SetCurrency(CurrencyID, Currency) -> None:
def SetCurrencyList(List[Currency]) -> None:
```

Сайт набирает популярность, мы снова задумываемся о масштабировании, снова развертываем десять серверов, но... Теперь сам вид наших интерфейсов совершенно отчетливо напоминает нам о том, что стоит масштабировать только интерфейс чтения, в то время как интерфейс записи, доступный исключительно администраторам и служебному ПО, обновляющему информацию на сайте — в масштабировании не нуждается, его вполне можно, как и раньше, запускать на одном-единственном сервере. Это не было секретом и раньше, при работе с первым, совместным интерфейсом, но представление об «общем интерфейсе» и «общих данных» настолько затуманило эту мысль, что вылилось уже в избыточные траты на масштабирование не нуждающегося в этом функционала.

Принцип CQRS приносит осознание того, что обработка команд и запросов бывает принципиально ассиметричной, а симметричное масштабирование сервисов часто не имеет никакого смысла.

Резюмируя, можно сказать, что сам по себе принцип CQRS достаточно банален, просто давая некий ментальный толчок к рассуждению об архитектуре приложения; сложности (и преимущества) будут лежать скорее в области архитектурных решений, выбранных вами для интеграции двух разделённых при помощи CQRS потоков данных.

### Практики

Agile


Scrum


Kanban


Какие есть семь этапов разработки продукта в Software Development lifecycle 

Определение требований
Проектирование
Конструирование (также «реализация» либо «кодирование»)
Воплощение
Тестирование и отладка (также «верификация»)
Инсталляция
Поддержка

Микросервисы  
https://habr.com/ru/post/249183/

Messaging

В этом мы рассмотрим системы асинхронной передачи сообщений (asynchronous message-passing system) — нечто среднее между RPC и базами данных. Они схожи с RPC в том, что запрос от клиента (обычно называемый сообщением (message)) доставляется другому процессу без особой задержки. Они напоминают БД тем, что сообщение не отправляется непосредственно через
сетевое соединение, а проходит через посредника, именуемого брокером сообщений (message broker), или очередью сообщений (message queue), или промежуточным ПО, ориентированным на обработку сообщений (message-oriented middleware), который временно хранит сообщение.

Использование брокера сообщений имеет несколько преимуществ по сравнению с RPC:  
он служит в качестве буфера в случае недоступности или перегруженности получателя, а следовательно, повышает надежность системы;  
он может автоматически отправлять сообщения повторно сбойным процессам, тем самым предотвращая потерю сообщений;  
благодаря ему отправителю не требуется знать IP-адрес и номера порта получателя (что особенно удобно в облачной среде, где виртуальные машины часто сменяют друг друга);  
он обеспечивает возможность отправки одного сообщения нескольким получателям;  
он логически расцепляет отправителя с получателем (отправитель лишь публикует сообщения, его не волнует, кто их получит).  

В отличие от RPC, описанное взаимодействие — как правило, одностороннее: отправитель обычно не ждет ответа на свои сообщения. Процессы могут отправлять ответы, но обычно задействуют отдельные каналы. Это асинхронный паттерн обмена сообщениями: отправитель не ждет доставки сообщения, а просто отправляет его и забывает о нем.

RabbitMQ  
Apache Kafka  
NATS  

Нюансы модели доставки зависят от реализации и конфигурации, но в целом брокеры сообщений используются следующим образом: один процесс отправляет сообщение, предназначенное для определенной очереди (queue) или дискуссии (topic), а брокер обеспечивает доставку данного сообщения одному или нескольким потребителям (consumers) или подписчикам (subscribers) этой очереди или дискуссии. В одной дискуссии может быть много инициаторов (producers) и много потребителей сообщений.

Дискуссия обеспечивает лишь односторонний поток данных. Однако потребитель способен сам опубликовать сообщения в другой дискуссии (так что можно будет их связать в цепочку) или в очередь ответов, потребляемых отправителем первоначального сообщения (это позволяет формировать поток запросов/ответов аналогично RPC).

Брокеры сообщений обычно не навязывают какой-либо конкретной модели данных — сообщение представляет собой просто байтовую последовательность с метаданными, так что можно использовать любой формат кодирования. Если ваш формат кодирования как прямо, так и обратно совместим, то у вас появляется возможность совершенно свободно менять издателей и потребителей сообщений независимо друг от друга и развертывать их в любом порядке.

Если потребитель публикует сообщение снова в другой дискуссии, то следует соблюдать осторожность и сохранять неизвестные поля, чтобы избежать проблем, связанных с новыми версиями данных.

https://habr.com/ru/company/innotech/blog/698838/

Цитата из статьи:

Выбирая между Kafka и RabbitMQ  
На самом деле, категорично сравнивать брокеры сообщений очень сложно. У всех существуют свои задачи и области применения. В случае с Apache Kafka и RabbitMQ это немного разный уровень, где лучшего не существует.

Kafka используется для обработки больших объёмов данных, сотен тысяч сообщений в секунду, которые подолгу хранятся на диске и много раз читаются сотнями или даже тысячами подписчиков. Kafka — это легко масштабируемая система, обладающая повышенной отказоустойчивостью, что очень важно в крупных проектах.

RabbitMQ более простой в установке и настройке, успешно справляется с асинхронным обменом данными в микросервисной архитектуре. Не требует дополнительных компонентов и затрат на дисковые ресурсы, так как все сообщения после чтения из очереди удаляются. По сравнению с Kafka обладает большими возможностями по настройке шаблонов обмена сообщениями. Отличный выбор, если нет завышенных требований к отказоустойчивости и пропускной способности.

Более того, сервер сам может служить клиентом для другого сервера (например, типичный веб-сервер приложений играет роль клиента по отношению к базе данных). Такой подход часто используется для декомпозиции больших приложений на небольшие сервисы по областям функциональности, чтобы сервисы выполняли запросы друг к другу в случае потребности в какой-либо функциональности или данных. Подобный метод создания приложений традиционно назывался сервисориентированной архитектурой (service-oriented architecture, SOA), а в последнее время подвергся пересмотру и получил новое название: микросервисная архитектура (microservices architecture).

В некоторых аспектах сервисы напоминают БД: они обычно предоставляют клиентам возможность отправлять и запрашивать данные. Однако в то время, как базы допускают произвольные запросы с помощью языков запросов, сервисы предоставляют API, свой для каждого приложения, допускающий только предопределенные бизнес-логикой (кодом приложения) сервиса входные и выходные данные. Это ограничение обеспечивает некую степень инкапсуляции: сервисы получают возможность в деталях ограничивать то, что разрешено и не разрешено делать клиентам.
Главная цель сервис-ориентированной и микросервисной архитектур — облегчить изменение и поддержку приложения путем обеспечения независимого развертывания и развития сервисов.  

Например, каждый из сервисов должен относиться к сфере ответственности одной команды разработчиков, которая должна быть в состоянии часто выпускать новые его версии, не нуждаясь в согласовании действий с другими командами. Другими словами, нужно быть готовыми к тому, что старые и новые версии серверов и клиентов будут работать параллельно, а значит, используемые серверами и клиентами форматы кодирования данных должны быть совместимы между разными версиями API сервиса.

### Теорема CAP

При проектировании распределенных систем необходимо понимать теорему CAP (Consistency, Availability, Partition Tolerance — «согласованность, доступность, устойчивость к секционированию»). Теорема CAP гласит, что распределенная система может обеспечивать не более двух из следующих трех свойств: согласованность, доступность и устойчивость к секционированию. Дадим несколько определений.
Согласованность. Означает, что все клиенты одновременно видят одни и те же данные, к какому бы узлу они ни подключились.  
Доступность. Означает, что любой клиент, запрашивающий данные, получает ответ, даже если некоторые из узлов недоступны.  
Устойчивость к секционированию. Секционирование свидетельствует о нарушении связи между двумя узлами. Устойчивость к секционированию означает, что система продолжает работать
вопреки нарушению связи в сети.  
Согласно теореме CAP, одним из этих свойств необходимо пожертвовать, чтобы обеспечить поддержку двух других.

Сейчас хранилища типа «ключ–значение» классифицируются в зависимости от того, какие две характеристики CAP они поддерживают:  
Системы CP (согласованность и устойчивость к секционированию). Жертвуют доступностью.  
Системы AP (доступность и устойчивость к секционированию). Жертвуют согласованностью.  
Системы CA (согласованность и доступность). Жертвуют устойчивостью к секционированию. Поскольку сетевые сбои неизбежны, распределенные системы должны справляться с разделением сети.  
В связи с этим системы типа CA не работоспособны в реальных условиях.

### Модели согласованности

Модель согласованности — еще один важный фактор, который следует учитывать при проектировании хранилища типа «ключ–значение». Она определяет степень согласованности данных и имеет широкий спектр
разновидностей.

Строгая согласованность. Любая операция чтения возвращает значение, соответствующее результату самой последней операции записи. Клиент всегда получает актуальные данные.  

Слабая согласованность. Последующие операции чтения могут и не вернуть самое последнее значение.  

Согласованность в конечном счете. Это разновидность слабой согласованности. Рано или поздно все обновления распространяются по системе и все реплики становятся согласованными.  

Жесткая согласованность обычно достигается за счет того, что операции чтения/записи принимаются только после подтверждения текущей записи всеми репликами. Это не самый оптимальный подход для высокодоступных систем, так как он может блокировать новые операции.  

В Dynamo и Cassandra используется отложенная согласованность. Она допускает поступление в систему несогласованных значений, заставляя клиента их прочитать и согласовать.